In [1]:
from datetime import datetime
from log import log_process
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = 'https://web.archive.org/web/20230908091635%20/https://en.wikipedia.org/wiki/List_of_largest_banks'

data_path = './data/'
log_name = 'log.txt'
db_name = 'Banks.db'


In [24]:
# Function to extract data under the heading "By market capitalization"
html = requests.get(url).text
data = BeautifulSoup(html, 'html.parser')

tables_html = data.find_all('tbody')
tables_html

[<tbody><tr>
 <th data-sort-type="number">Rank
 </th>
 <th>Bank name
 </th>
 <th>Market cap<br/>(US$ billion)
 </th></tr>
 <tr>
 <td>1
 </td>
 <td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/United_States" title="United States"><img alt="United States" class="mw-file-element" data-file-height="650" data-file-width="1235" decoding="async" height="12" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/23px-Flag_of_the_United_States.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/35px-Flag_of_the_United_States.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/en/thumb/a/a4/Flag_of_the_United_States.svg/46px-Flag_of_the_United_States.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908

In [4]:
n_tables = len(tables_html)

table = tables_html[0]
columns = [col.contents[0].strip() for col in table.find_all('th')]
extracted_data = pd.DataFrame(columns=columns)

# table
rows = table.find_all('tr')

rows[-1]

<tr>
<td>10
</td>
<td><span class="flagicon"><span class="mw-image-border" typeof="mw:File"><a href="/web/20230908091635/https://en.wikipedia.org/wiki/China" title="China"><img alt="China" class="mw-file-element" data-file-height="600" data-file-width="900" decoding="async" height="15" src="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/23px-Flag_of_the_People%27s_Republic_of_China.svg.png" srcset="//web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/35px-Flag_of_the_People%27s_Republic_of_China.svg.png 1.5x, //web.archive.org/web/20230908091635im_/https://upload.wikimedia.org/wikipedia/commons/thumb/f/fa/Flag_of_the_People%27s_Republic_of_China.svg/45px-Flag_of_the_People%27s_Republic_of_China.svg.png 2x" width="23"/></a></span></span> <a href="/web/20230908091635/https://en.wikipedia.org/wiki/Bank_of_

In [5]:
for row in rows[1:len(rows)]:
  cols = row.find_all('td')
  _values = []
  for value in cols:
    # Some rows have a different structure
    try:
      _values.append(value.contents[0].strip())
    except:
      _values.append(value.find_all('a')[1].contents[0].strip())
  _dict = {columns[i]: value for i,value in enumerate(_values)}
  _df = pd.DataFrame(_dict, index=[0])
  extracted_data = pd.concat([extracted_data,_df], ignore_index=True)
  
display(extracted_data)

,Rank,Bank name,Market cap
0,1,JPMorgan Chase,432.92
1,2,Bank of America,231.52
2,3,Industrial and Commercial Bank of China,194.56
3,4,Agricultural Bank of China,160.68
4,5,HDFC Bank,157.91
5,6,Wells Fargo,155.87
6,7,HSBC Holdings PLC,148.90
7,8,Morgan Stanley,140.83
8,9,China Construction Bank,139.82
9,10,Bank of China,136.81


In [23]:
exchange_rate = pd.read_csv('./data/exchange_rate.csv')
extracted_data['Market cap EUR'] = extracted_data['Market cap'].values.astype('float') * exchange_rate.query("Currency == 'EUR'")['Rate'].values[0]
extracted_data

,Rank,Bank name,Market cap,Market cap GBP,Market cap EUR
0,1,JPMorgan Chase,432.92,346.336,402.6156
1,2,Bank of America,231.52,185.216,215.3136
2,3,Industrial and Commercial Bank of China,194.56,155.648,180.9408
3,4,Agricultural Bank of China,160.68,128.544,149.4324
4,5,HDFC Bank,157.91,126.328,146.8563
5,6,Wells Fargo,155.87,124.696,144.9591
6,7,HSBC Holdings PLC,148.90,119.120,138.4770
7,8,Morgan Stanley,140.83,112.664,130.9719
8,9,China Construction Bank,139.82,111.856,130.0326
9,10,Bank of China,136.81,109.448,127.2333
